In [5]:
import xeno as x
import csv
import joblib
import pandas as pd

In [6]:
model = joblib.load(f'xeno_xgbregressor_30.pkl')

In [7]:
# cpuの行動を定義
def choice_cpu(now,choices,kind):
    scores = {}
    for choice in choices:
        data = {}


        card_number = now['card_number']
        my_hands = now['my_hands']
        my_played = now['my_played']
        other_played = now['other_played']
        look_hands = now['look_hands']
        looked_hands = now['looked_hands']
        pred_data = now['pred']
        reincarnation = int(now['reincarnation'])
        
        data['card_number'] = int(card_number)
        for j in range(2):
            if j < len(my_hands):
                data[f'my_hands_{j}'] = int(my_hands[j])
            else:
                data[f'my_hands_{j}'] = 0

        for j in range(15):
            if j < len(my_played):
                data[f'my_played_{j}'] = int(my_played[j])
            else:
                data[f'my_played_{j}'] = 0
        
        for j in range(now['players_length']):
            j += 1
            for k in range(15):
                if j in list(other_played.keys()):
                    if k < len(other_played[j]):
                        data[f'other_played_{j}_{k}'] = int(other_played[j][k])
                    else:
                        data[f'other_played_{j}_{k}'] = 0
                else:
                    data[f'other_played_{j}_{k}'] = 0
            
            for k in range(15):
                if j in list(other_played.keys()):
                    if k < len(look_hands[j]):
                        data[f'look_hands_{j}_{k}'] = int(look_hands[j][k])
                    else:
                        data[f'look_hands_{j}_{k}'] = 0
                else:
                    data[f'look_hands_{j}_{k}'] = 0
            
            for k in range(15):
                if j in list(look_hands.keys()):
                    if k < len(looked_hands[j]):
                        data[f'looked_hands_{j}_{k}'] = int(looked_hands[j][k])
                    else:
                        data[f'looked_hands_{j}_{k}'] = 0
                else:
                    data[f'looked_hands_{j}_{k}'] = 0
            
        for k in range(2):
            if k < len(pred_data):
                data[f'pred_{k}_subject'] = int(pred_data[k]['subject'])
                data[f'pred_{k}_object'] = int(pred_data[k]['object'])
                data[f'pred_{k}_card'] = int(pred_data[k]['pred_card'])
            else:
                data[f'pred_{k}_subject'] = 0
                data[f'pred_{k}_object'] = 0
                data[f'pred_{k}_card'] = 0
        
        data['reincarnation'] = reincarnation
        data['kind'] = kind
        for i in range(10):
            if i < len(choices):
                if kind=='opponentChoice':
                    data[f'choices_{i}'] = choices[i]['player'].turn_number
                else:
                    data[f'choices_{i}'] = choices[i]
            else:
                data[f'choices_{i}'] = 0

        if kind=='opponentChoice':
            data['choice'] = choice['select_number']
        else:
            data['choice'] = choice
        df = pd.DataFrame(data=data,index=[0])
        
        for i in range(len(list(df['kind']))):
            if df.iloc[i]['kind'] == 'draw':
                df.at[i,'kind'] = 0
            elif df.iloc[i]['kind'] == 'opponentChoice':
                df.at[i,'kind'] = 1
            elif df.iloc[i]['kind'] == 'play_card':
                df.at[i,'kind'] = 2
            elif df.iloc[i]['kind'] == 'pred':
                df.at[i,'kind'] = 3
            elif df.iloc[i]['kind'] == 'trush':
                df.at[i,'kind'] = 4

        df.astype(float)
        x = df.values
        print(now)
        print(choices)
        print(kind)
        print(df)
        print(x)
        score = model.predict(x)
        if kind == 'opponentChoice':
            choice_name = choice['select_number']
            scores[f'{choice_name}'] = score
        else:
            scores[f'{choice}'] = score
    select = max(scores, key=scores.get)
    return select

In [8]:
# cpuの命名規則を定義
def get_name_cpu(index:int):
    return 'test_player'+str(index)

In [9]:
funcs = [{'get_name':get_name_cpu, 'choice':choice_cpu},{'get_name':get_name_cpu,'choice':choice_cpu}]

In [10]:
for _ in range(10**0):
    game = x.Game(2, funcs=funcs)
    data = game.game()
    if data[0]:
        log = data[1]
        with open('result/log.csv', 'a') as f:
            writer = csv.writer(f)
            for l in log:
                writer.writerow(l)
                result = l[-1]
                if (result != 'win') and (result != 'lose'):
                    print('勝利判定がなされていません')
                    break
        
    else:
        log = data[2]
        err = data[1]
        print(err)
        with open('result/log.csv', 'a') as f:
            writer = csv.writer(f)
            for l in log:
                writer.writerow(l)
        break

{'players_length': 2, 'my_turn_number': 1, 'other_players': {2: True}, 'card_number': 17, 'my_hands': [], 'my_played': [], 'other_played': {2: []}, 'look_hands': {2: []}, 'looked_hands': {2: []}, 'pred': [], 'reincarnation': True}
[1]
draw
   card_number  my_hands_0  my_hands_1  my_played_0  my_played_1  my_played_2  \
0           17           0           0            0            0            0   

   my_played_3  my_played_4  my_played_5  my_played_6  ...  choices_1  \
0            0            0            0            0  ...          0   

   choices_2  choices_3  choices_4  choices_5  choices_6  choices_7  \
0          0          0          0          0          0          0   

   choices_8  choices_9  choice  
0          0          0       1  

[1 rows x 127 columns]
[[17 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 